In [1]:
import pandas as pd
import numpy as np
import timeit
import matplotlib.pyplot as plt
import json
import re

In [2]:
filepath = "/Users/Chrisdon/Documents/Training/Python/data/"

### Load a compressed JSON file into a dataframe

In [3]:
try:
   recipes = pd.read_json(filepath+'recipe_data.zip', compression="zip")
except ValueError as e:
   print("ValueError:", e)
recipes.head()

,_id,name,ingredients,url,image,ts,cookTime,source,recipeYield,datePublished,prepTime,description,totalTime,creator,recipeCategory,dateModified,recipeInstructions
0,{'$oid': '516c77f796cc62548fd30a5c'},Tangy Fajita Marinade,1 cup pineapple juice\n1/2 cup soy sauce\n2 ta...,http://allrecipes.com/Recipe/Tangy-Fajita-Mari...,http://images.media-allrecipes.com/userphotos/...,{'$date': 1366063095269},None,allrecipes,1 1/2 cups,None,PT5M,"""This is the best marinade I have ever tasted....",None,None,None,None,NaN
1,{'$oid': '5161f2da96cc620d2615c636'},Healthy Chocolate Chip Oatmeal Cookies,¼ cups ¼ cups\n½ cups ½ cups\n1 whole 1 whole\...,http://tastykitchen.com/recipes/desserts/healt...,http://static.tastykitchen.com/recipes/files/2...,{'$date': 1365373658520},PT10M,tastykitchen,12,2011-04-05,PT15M,These tasty oatmeal chocolate chip cookies are...,None,None,None,None,NaN
2,{'$oid': '516a062a96cc624c4d8f009b'},Courgette and Parmesan Soup,"1 large onion, chopped\n2 gloves garlic, slice...",http://www.bbcgoodfood.com/recipes/2535636/cou...,None,{'$date': 1365902890901},None,bbcgoodfood,Serves 4,None,None,"fresh summer soup, perfect for using up a glut...",None,None,None,None,NaN
3,{'$oid': '516077d596cc6208b33001e2'},Shrimpo de Gallo,"1 pound Cooked Shrimp (tails Removed), Chopped...",http://thepioneerwoman.com/cooking/2011/01/shr...,http://static.thepioneerwoman.com/cooking/file...,{'$date': 1365276629819},PT,thepioneerwoman,12,2011-01-26,PT15M,"Yes, I made up the name for this recipe. Why d...",None,None,None,None,NaN
4,{'$oid': '516a062e96cc624c4d8f00a5'},Courgette &amp; watercress salad with grilled ...,about 12 baby courgettes\nolive oil\n4 fillets...,http://www.bbcgoodfood.com/recipes/6672/courge...,http://www.bbcgoodfood.com/recipes/6672/images...,{'$date': 1365902894765},PT10M,bbcgoodfood,Serves 4,None,PT10M,Cooking fish and teaming it with salad is perf...,None,None,None,None,NaN


In [7]:
recipes.iloc[0] # examine the first entry in the dataframe
recipes.loc[0:2,"name"]

0                     Tangy Fajita Marinade
1    Healthy Chocolate Chip Oatmeal Cookies
2               Courgette and Parmesan Soup
Name: name, dtype: object

In [5]:
# The ingredient lists average 250 characters long, with a minimum of 0 and a maximum of nearly 10,000 characters!
recipes.ingredients.str.len().describe()

count    8664.000000
mean      243.371191
std       147.815138
min         1.000000
25%       147.000000
50%       219.000000
75%       313.000000
max      3247.000000
Name: ingredients, dtype: float64

In [6]:
recipes.name[np.argmax(recipes.ingredients.str.len())] # which recipe has the longest ingredient list:

'Braised Beef cheeks Recipe'

In [7]:
recipes.description.str.contains('[Bb]reakfast').sum() # how many of the recipes are for breakfast food:

179

In [8]:
# how many of the recipes list cinnamon as an ingredient:
recipes.ingredients.str.contains('[Cc]innamon').sum()

526

## Simple recipe recommendation system

In [9]:
spice_list = ['salt', 'pepper', 'oregano', 'sage', 'parsley',
 'rosemary', 'tarragon', 'thyme', 'paprika', 'cumin']
# build a Boolean DataFrame consisting of True and False values, indicating whether this ingredient appears in the list
spice_df = pd.DataFrame(
 dict((spice, recipes.ingredients.str.contains(spice, re.IGNORECASE))
 for spice in spice_list))
spice_df.head()

,salt,pepper,oregano,sage,parsley,rosemary,tarragon,thyme,paprika,cumin
0,False,False,False,False,False,False,False,False,False,True
1,False,False,False,False,False,False,False,False,False,False
2,False,False,False,False,False,False,False,False,False,False
3,False,False,False,False,False,False,False,False,False,False
4,False,False,False,False,True,False,False,False,False,False


In [10]:
# find a recipe that uses parsley, paprika, and tarragon
selection = spice_df.query('parsley & paprika & tarragon')
len(selection)
#We find only 10 recipes with this combination; let’s use the index returned by this
#selection to discover the names of the recipes that have this combination:
recipes.name[selection.index]

4681    Lamb shank tagine with herb tabbouleh
Name: name, dtype: object